In [1]:
import tensorflow.keras.datasets.mnist as mnist
import numpy as np
import matplotlib.pyplot as plt
import time

import sys
import os
sys.path.insert(0, os.path.abspath("../"))

from core.label_utils import label_binerizer

### Import and prepare dataset

In [2]:
BATCH_NORM_MOMENTUM=0.9
BATCH_SIZE = 100
LR_ADAM=1e-3 # from paper pg 18: https://openreview.net/pdf?id=pwwVuSICBgt
LR_SGD=1e-1 # from paper pg 18: https://openreview.net/pdf?id=pwwVuSICBgt
DTYPE = np.float32

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = np.expand_dims(X_train.astype(DTYPE), -1)/255
y_train = label_binerizer(y_train.astype(int), n_class=10)
X_test = np.expand_dims(X_test, -1).astype(DTYPE)/255
y_test = label_binerizer(y_test.astype(int), n_class=10)

idx_choice = list(range(len(X_test)))

print("Training samples: ", X_train.shape)
print("Testing samples: ", X_test.shape)

Training samples:  (60000, 28, 28, 1)
Testing samples:  (10000, 28, 28, 1)


### Numpy full model implementation

### Binary model

In [4]:
#del sys.modules["core.binary_models.layers"]
from core.numpy_models.full_precision_models.layers import Flatten, Softmax
from core.losses import CrossEntropy
from core.optimizers import GradientDescent, Adam
from core.label_utils import label_binerizer

from core.numpy_models.binary_models.layers import BinaryDense, BinaryConv2D#, BatchNorm
from core.numpy_models.xnor_models.layers import  BatchNorm
from core.numpy_models.binary_models.activation_layers import BinaryActivation
from core.model import Sequential

In [5]:
### Single conv + flatten + dense layer


model_layers = [
    Flatten(), 
    BinaryDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    BinaryDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    BinaryDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    BinaryDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    BinaryDense(10, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    Softmax()
]

loss_op = CrossEntropy()
optimizer_op = GradientDescent(lr=LR_SGD)

LeNet5_numpy = Sequential(model_layers)
LeNet5_numpy.compile(loss_op, optimizer_op)


for epoch in range(100):
    loss_batch = np.array([])
    acc_batch = np.array([])
    start_time = time.time()
    for j in range(len(X_train)//BATCH_SIZE):
        print(f"Batch {j}", end="\r")
        X_batch = X_train[j*BATCH_SIZE:(j+1)*BATCH_SIZE]
        y_batch = y_train[j*BATCH_SIZE:(j+1)*BATCH_SIZE]
        LeNet5_numpy.fit_step(X_batch, y_batch, curr_step=epoch)
        loss_batch = np.append(loss_batch, LeNet5_numpy.loss)
        acc_batch = np.append(acc_batch, LeNet5_numpy.accuracy)
    end_time = time.time()
    rnd_idx = np.random.choice(idx_choice, size=5000, replace=False)
    LeNet5_numpy.validate_step(X=X_test[rnd_idx], y=y_test[rnd_idx], curr_step=epoch)
    print(f"Epoch {epoch}, training time = {end_time-start_time} s:") 
    print(f"Loss = {np.mean(loss_batch)}, Accuracy = {np.mean(acc_batch)}")
    print(f"Validation loss = {LeNet5_numpy.validation_loss}, Validation accuracy = {LeNet5_numpy.validation_accuracy}")

Epoch 0, training time = 20.13914918899536 s:
Loss = 0.4307805379231771, Accuracy = 0.8940666666666668
Validation loss = 0.29248046875, Validation accuracy = 0.9256
Epoch 1, training time = 19.85128140449524 s:
Loss = 0.28419336954752605, Accuracy = 0.9331999999999999
Validation loss = 0.24609375, Validation accuracy = 0.9382
Epoch 2, training time = 20.064897537231445 s:
Loss = 0.24467595418294272, Accuracy = 0.9442833333333333
Validation loss = 0.2261962890625, Validation accuracy = 0.9484
Epoch 3, training time = 19.58268404006958 s:
Loss = 0.2242730712890625, Accuracy = 0.9518833333333333
Validation loss = 0.2103271484375, Validation accuracy = 0.95
Epoch 4, training time = 19.684598207473755 s:
Loss = 0.20936335245768228, Accuracy = 0.9558333333333333
Validation loss = 0.2054443359375, Validation accuracy = 0.953
Epoch 5, training time = 19.978805541992188 s:
Loss = 0.19917872111002605, Accuracy = 0.9593
Validation loss = 0.1968994140625, Validation accuracy = 0.9522
Epoch 6, trai

Epoch 50, training time = 20.02234411239624 s:
Loss = 0.13238006591796875, Accuracy = 0.9808166666666667
Validation loss = 0.1676025390625, Validation accuracy = 0.965
Epoch 51, training time = 20.85740327835083 s:
Loss = 0.13222081502278646, Accuracy = 0.9811166666666666
Validation loss = 0.15625, Validation accuracy = 0.967
Epoch 52, training time = 20.84971046447754 s:
Loss = 0.13081044514973958, Accuracy = 0.9815000000000002
Validation loss = 0.1702880859375, Validation accuracy = 0.9634
Epoch 53, training time = 20.35329246520996 s:
Loss = 0.13233322143554688, Accuracy = 0.9807833333333333
Validation loss = 0.1690673828125, Validation accuracy = 0.963
Epoch 54, training time = 20.011410236358643 s:
Loss = 0.1309161376953125, Accuracy = 0.9811333333333334
Validation loss = 0.1707763671875, Validation accuracy = 0.9642
Epoch 55, training time = 19.95920729637146 s:
Loss = 0.13011144002278646, Accuracy = 0.9816166666666667
Validation loss = 0.158203125, Validation accuracy = 0.9658
E

In [6]:
### Single conv + flatten + dense layer


model_layers = [
    Flatten(), 
    BinaryDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    BinaryDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    BinaryDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    BinaryDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    BinaryDense(10, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    Softmax()
]

loss_op = CrossEntropy()
optimizer_op = Adam(lr=LR_ADAM)

LeNet5_numpy = Sequential(model_layers)
LeNet5_numpy.compile(loss_op, optimizer_op)

for epoch in range(100):
    loss_batch = np.array([])
    acc_batch = np.array([])
    start_time = time.time()
    for j in range(len(X_train)//BATCH_SIZE):
        print(f"Batch {j}", end="\r")
        X_batch = X_train[j*BATCH_SIZE:(j+1)*BATCH_SIZE]
        y_batch = y_train[j*BATCH_SIZE:(j+1)*BATCH_SIZE]
        LeNet5_numpy.fit_step(X_batch, y_batch, curr_step=epoch)
        loss_batch = np.append(loss_batch, LeNet5_numpy.loss)
        acc_batch = np.append(acc_batch, LeNet5_numpy.accuracy)
    end_time = time.time()
    rnd_idx = np.random.choice(idx_choice, size=5000, replace=False)
    LeNet5_numpy.validate_step(X=X_test[rnd_idx], y=y_test[rnd_idx], curr_step=epoch)
    print(f"Epoch {epoch}, training time = {end_time-start_time} s:") 
    print(f"Loss = {np.mean(loss_batch)}, Accuracy = {np.mean(acc_batch)}")
    print(f"Validation loss = {LeNet5_numpy.validation_loss}, Validation accuracy = {LeNet5_numpy.validation_accuracy}")

Epoch 0, training time = 33.270968437194824 s:
Loss = 0.4378831990559896, Accuracy = 0.8915666666666667
Validation loss = 0.2734375, Validation accuracy = 0.93
Epoch 1, training time = 33.25434637069702 s:
Loss = 0.28442525227864585, Accuracy = 0.9329999999999999
Validation loss = 0.2393798828125, Validation accuracy = 0.9426
Epoch 2, training time = 34.39118027687073 s:
Loss = 0.24460047403971355, Accuracy = 0.9451499999999998
Validation loss = 0.2178955078125, Validation accuracy = 0.951
Epoch 3, training time = 36.24670648574829 s:
Loss = 0.2259118143717448, Accuracy = 0.9505166666666666
Validation loss = 0.1971435546875, Validation accuracy = 0.954
Epoch 4, training time = 39.48231267929077 s:
Loss = 0.21488505045572917, Accuracy = 0.9543166666666667
Validation loss = 0.1988525390625, Validation accuracy = 0.9546
Epoch 5, training time = 40.50728940963745 s:
Loss = 0.2037714131673177, Accuracy = 0.9572333333333332
Validation loss = 0.1912841796875, Validation accuracy = 0.9546
Epoc

Epoch 50, training time = 33.291762351989746 s:
Loss = 0.13910018920898437, Accuracy = 0.9787333333333333
Validation loss = 0.1610107421875, Validation accuracy = 0.965
Epoch 51, training time = 33.449650287628174 s:
Loss = 0.1365460713704427, Accuracy = 0.9799166666666668
Validation loss = 0.1595458984375, Validation accuracy = 0.9632
Epoch 52, training time = 34.71595072746277 s:
Loss = 0.13788828531901043, Accuracy = 0.9794500000000002
Validation loss = 0.1607666015625, Validation accuracy = 0.9638
Epoch 53, training time = 40.01061677932739 s:
Loss = 0.13796579996744793, Accuracy = 0.9796
Validation loss = 0.1734619140625, Validation accuracy = 0.9612
Epoch 54, training time = 40.90952658653259 s:
Loss = 0.1371533711751302, Accuracy = 0.9800166666666666
Validation loss = 0.17578125, Validation accuracy = 0.9612
Epoch 55, training time = 41.22816801071167 s:
Loss = 0.13724756876627603, Accuracy = 0.9798833333333332
Validation loss = 0.16015625, Validation accuracy = 0.9646
Epoch 56,

### XNor Models

In [6]:
#del sys.modules["core.binary_models.layers"]
from core.numpy_models.full_precision_models.layers import Flatten, Softmax
from core.losses import CrossEntropy
from core.optimizers import GradientDescent, Adam
from core.label_utils import label_binerizer

from core.numpy_models.xnor_models.layers import XNorDense, XNorConv2D, BatchNorm
from core.numpy_models.binary_models.activation_layers import BinaryActivation
from core.model import Sequential

In [8]:
### Single conv + flatten + dense layer


model_layers = [
    Flatten(), 
    XNorDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    XNorDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    XNorDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    XNorDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    XNorDense(10, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    Softmax()
]

loss_op = CrossEntropy()
optimizer_op = GradientDescent(lr=LR_SGD)

LeNet5_numpy = Sequential(model_layers)
LeNet5_numpy.compile(loss_op, optimizer_op)

for epoch in range(100):
    loss_batch = np.array([])
    acc_batch = np.array([])
    start_time = time.time()
    for j in range(len(X_train)//BATCH_SIZE):
        print(f"Batch {j}", end="\r")
        X_batch = X_train[j*BATCH_SIZE:(j+1)*BATCH_SIZE]
        y_batch = y_train[j*BATCH_SIZE:(j+1)*BATCH_SIZE]
        LeNet5_numpy.fit_step(X_batch, y_batch, curr_step=epoch)
        loss_batch = np.append(loss_batch, LeNet5_numpy.loss)
        acc_batch = np.append(acc_batch, LeNet5_numpy.accuracy)
    end_time = time.time()
    LeNet5_numpy.validate_step(X=X_test[:2000], y=y_test[:2000], curr_step=epoch)
    print(f"Epoch {epoch}, training time = {end_time-start_time} s:") 
    print(f"Loss = {np.mean(loss_batch)}, Accuracy = {np.mean(acc_batch)}")
    print(f"Validation loss = {LeNet5_numpy.validation_loss}, Validation accuracy = {LeNet5_numpy.validation_accuracy}")

Epoch 0: Loss = 0.6165789794921875, Accuracy = 0.8359333333333332
Validation loss = 0.45361328125, Validation accuracy = 0.8775
Epoch 1: Loss = 0.3451017761230469, Accuracy = 0.9127666666666666
Validation loss = 0.37353515625, Validation accuracy = 0.9
Epoch 2: Loss = 0.3038633728027344, Accuracy = 0.9246833333333332
Validation loss = 0.345703125, Validation accuracy = 0.9105
Epoch 3: Loss = 0.27795491536458333, Accuracy = 0.9336166666666667
Validation loss = 0.32080078125, Validation accuracy = 0.912
Epoch 4: Loss = 0.263067372639974, Accuracy = 0.9383333333333334
Validation loss = 0.307373046875, Validation accuracy = 0.915
Epoch 5: Loss = 0.24993306477864582, Accuracy = 0.9428833333333334
Validation loss = 0.288330078125, Validation accuracy = 0.921
Epoch 6: Loss = 0.24060007731119792, Accuracy = 0.9454333333333333
Validation loss = 0.271240234375, Validation accuracy = 0.9285
Epoch 7: Loss = 0.23425633748372396, Accuracy = 0.9477999999999999
Validation loss = 0.271484375, Validatio

Epoch 65: Loss = 0.1752697245279948, Accuracy = 0.9674333333333334
Validation loss = 0.2242431640625, Validation accuracy = 0.9475
Epoch 66: Loss = 0.17621729532877603, Accuracy = 0.9675666666666666
Validation loss = 0.2265625, Validation accuracy = 0.945
Epoch 67: Loss = 0.17697057088216145, Accuracy = 0.9665833333333332
Validation loss = 0.21826171875, Validation accuracy = 0.947
Epoch 68: Loss = 0.17656600952148438, Accuracy = 0.9670666666666666
Validation loss = 0.2161865234375, Validation accuracy = 0.9495
Epoch 69: Loss = 0.17581929524739584, Accuracy = 0.9669333333333334
Validation loss = 0.2257080078125, Validation accuracy = 0.946
Epoch 70: Loss = 0.1765485127766927, Accuracy = 0.9665333333333332
Validation loss = 0.2158203125, Validation accuracy = 0.951
Epoch 71: Loss = 0.1771350606282552, Accuracy = 0.9670166666666667
Validation loss = 0.2117919921875, Validation accuracy = 0.9485
Epoch 72: Loss = 0.175999755859375, Accuracy = 0.9670666666666666
Validation loss = 0.21655273

In [9]:
### Single conv + flatten + dense layer


model_layers = [
    Flatten(), 
    XNorDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    XNorDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    XNorDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    XNorDense(256, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    BinaryActivation(dtype=DTYPE),
    XNorDense(10, dtype=DTYPE),
    BatchNorm(BATCH_NORM_MOMENTUM),
    Softmax()
]

loss_op = CrossEntropy()
optimizer_op = Adam(lr=LR_ADAM)

LeNet5_numpy = Sequential(model_layers)
LeNet5_numpy.compile(loss_op, optimizer_op)

for epoch in range(100):
    loss_batch = np.array([])
    acc_batch = np.array([])
    start_time = time.time()
    for j in range(len(X_train)//BATCH_SIZE):
        print(f"Batch {j}", end="\r")
        X_batch = X_train[j*BATCH_SIZE:(j+1)*BATCH_SIZE]
        y_batch = y_train[j*BATCH_SIZE:(j+1)*BATCH_SIZE]
        LeNet5_numpy.fit_step(X_batch, y_batch, curr_step=epoch)
        loss_batch = np.append(loss_batch, LeNet5_numpy.loss)
        acc_batch = np.append(acc_batch, LeNet5_numpy.accuracy)
    end_time = time.time()
    LeNet5_numpy.validate_step(X=X_test[:2000], y=y_test[:2000], curr_step=epoch)
    print(f"Epoch {epoch}, training time = {end_time-start_time} s:") 
    print(f"Loss = {np.mean(loss_batch)}, Accuracy = {np.mean(acc_batch)}")
    print(f"Validation loss = {LeNet5_numpy.validation_loss}, Validation accuracy = {LeNet5_numpy.validation_accuracy}")

Epoch 0: Loss = 0.42358184814453126, Accuracy = 0.8899333333333334
Validation loss = 0.359375, Validation accuracy = 0.9045
Epoch 1: Loss = 0.29525146484375, Accuracy = 0.9285333333333333
Validation loss = 0.321533203125, Validation accuracy = 0.9225
Epoch 2: Loss = 0.26822703043619794, Accuracy = 0.9368166666666667
Validation loss = 0.296630859375, Validation accuracy = 0.9265
Epoch 3: Loss = 0.2532208760579427, Accuracy = 0.9410333333333334
Validation loss = 0.28564453125, Validation accuracy = 0.9275
Epoch 4: Loss = 0.24454208374023437, Accuracy = 0.9443999999999998
Validation loss = 0.267333984375, Validation accuracy = 0.93
Epoch 5: Loss = 0.23999287923177084, Accuracy = 0.9449833333333334
Validation loss = 0.261474609375, Validation accuracy = 0.934
Epoch 6: Loss = 0.23208094278971353, Accuracy = 0.9489833333333333
Validation loss = 0.269287109375, Validation accuracy = 0.931
Epoch 7: Loss = 0.23028477986653645, Accuracy = 0.9480499999999998
Validation loss = 0.26220703125, Valid

Epoch 65: Loss = 0.18758941650390626, Accuracy = 0.96295
Validation loss = 0.224365234375, Validation accuracy = 0.9485
Epoch 66: Loss = 0.18718582153320312, Accuracy = 0.9634833333333334
Validation loss = 0.2177734375, Validation accuracy = 0.948
Epoch 67: Loss = 0.18789077758789063, Accuracy = 0.9634833333333332
Validation loss = 0.223388671875, Validation accuracy = 0.945
Epoch 68: Loss = 0.18757766723632813, Accuracy = 0.9633833333333333
Validation loss = 0.2119140625, Validation accuracy = 0.948
Epoch 69: Loss = 0.18877644856770834, Accuracy = 0.9628333333333332
Validation loss = 0.2177734375, Validation accuracy = 0.9465
Epoch 70: Loss = 0.18746480305989582, Accuracy = 0.9633333333333334
Validation loss = 0.233642578125, Validation accuracy = 0.949
Epoch 71: Loss = 0.18538162231445313, Accuracy = 0.96375
Validation loss = 0.2265625, Validation accuracy = 0.945
Epoch 72: Loss = 0.18605489095052083, Accuracy = 0.9637666666666667
Validation loss = 0.231201171875, Validation accuracy